<a href="https://colab.research.google.com/github/lvlanson/data_mining/blob/main/Praktikum_6_Aufgabe_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Aufgabe 3**

Erstellen Sie ein Naive Bayes und ein k-NN Verfahren für den Iris Datensatz und erzeugen Sie das ROC Diagramm für beide Modelle.

In [310]:
from sklearn.datasets import load_iris

# Iris Datensatz in Attribute(X) und Labels(y) laden
X, y = load_iris(return_X_y=True)

labels = load_iris().feature_names

In [311]:
# Helfer Funktionen zum Plotten

from plotly.subplots import make_subplots
from math import ceil, floor
import plotly.graph_objects as go

def plot_scatter_dataframe(df, target: int, title=""):
  fig = make_subplots(rows=1, cols=1, subplot_titles=(title, "Original"))
    
  columns = len(df.columns)
  for i in range(columns):
    if i == target:
      continue
    fig.add_trace(
      go.Scatter(
          x=df.iloc[:, i], 
          y=df.iloc[:, target],
          mode="markers",
          name=df.columns[i]
      )
    )
  
  fig.show()

def boxplot_dataframe(df, target: int, title=""):

  data = []
  for i, col in enumerate(df.columns):
    if i != target:
      trace = go.Box(y=df[col],
                    name=col)
      data.append(trace)


  fig = go.Figure(data=data)
  fig.update_layout(title_text=title)
  fig.show()


In [312]:
import pandas as pd
import matplotlib as plt

# Datensatz für Auswertung in Dataframe packen
df = pd.DataFrame(X, columns=labels)

# Labels hinzufügen
df.insert(4, "Labels", y)

# Dataframe analyisieren
pd.set_option('display.max_columns', None)
print(df.describe(include="all"))
boxplot_dataframe(df, 4)

       sepal length (cm)  sepal width (cm)  petal length (cm)  \
count         150.000000        150.000000         150.000000   
mean            5.843333          3.057333           3.758000   
std             0.828066          0.435866           1.765298   
min             4.300000          2.000000           1.000000   
25%             5.100000          2.800000           1.600000   
50%             5.800000          3.000000           4.350000   
75%             6.400000          3.300000           5.100000   
max             7.900000          4.400000           6.900000   

       petal width (cm)      Labels  
count        150.000000  150.000000  
mean           1.199333    1.000000  
std            0.762238    0.819232  
min            0.100000    0.000000  
25%            0.300000    0.000000  
50%            1.300000    1.000000  
75%            1.800000    2.000000  
max            2.500000    2.000000  


In [313]:
# Helfer Funktion zum Entfernen von Ausreißern

from sklearn.neighbors import LocalOutlierFactor
import numpy as np

def remove_outliers(df, n):

  detector = LocalOutlierFactor(n_neighbors=20)
  detector.fit(df)

  factors = detector.negative_outlier_factor_
  outliers_sorted = np.sort(factors)[:n]

  indices = []
  for outlier in outliers_sorted:
    indices.append(*np.where(factors==outlier)[0])

  df_outliers = df.iloc[indices, :]
  print(df_outliers.index)
  display(df_outliers)

  df_clean = df.drop(indices)
  df_clean = df_clean.dropna()
  return df_clean

In [314]:
# Ausreißer entfernen

df_clean = remove_outliers(df, 8)

boxplot_dataframe(df, 4, "Unbereinigt")
boxplot_dataframe(df_clean, 4, "Bereinigt")

Int64Index([41, 15, 106, 118, 117, 131, 98, 14], dtype='int64')


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Labels
41,4.5,2.3,1.3,0.3,0
15,5.7,4.4,1.5,0.4,0
106,4.9,2.5,4.5,1.7,2
118,7.7,2.6,6.9,2.3,2
117,7.7,3.8,6.7,2.2,2
131,7.9,3.8,6.4,2.0,2
98,5.1,2.5,3.0,1.1,1
14,5.8,4.0,1.2,0.2,0


In [315]:
from sklearn.preprocessing import QuantileTransformer

def normalize(df, target, n_quantiles=0):
  if n_quantiles==0:
    n_quantiles = len(df)

  # Labels entfernen
  
  df_attr = df.drop(columns=df.columns[target])
  df_labels = df.iloc[:, target:target+1]

  scaler = QuantileTransformer(n_quantiles=n_quantiles)
  scaler.fit(df_attr)

  df_qt = scaler.transform(df_attr)
  df_qt = pd.DataFrame(df_qt, columns=df_attr.columns)
  df_qt["Labels"] = df_labels.values
  return df_qt

In [316]:
# Daten normalisieren

df_norm = normalize(df_clean, 4)
plot_scatter_dataframe(df_clean, 4, "Un")
plot_scatter_dataframe(df_norm, 4, "Normalisiert")

In [317]:
# Helferfunktion zum Splitten der Daten

from sklearn.model_selection import train_test_split

def create_set(data_frame, test_size):
  train, test = train_test_split(df_rs, test_size=test_size)

  x_train = train.loc[:, data_frame.columns != "Gewicht_lbs"]
  y_train = train["Gewicht_lbs"]

  x_test = test.loc[:, data_frame.columns != "Gewicht_lbs"]
  y_test = test["Gewicht_lbs"]


  return (x_train, y_train), (x_test, y_test)

In [318]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

X = df_norm.iloc[:, 0:4].values
y = df_norm.iloc[:, 4:5].values

X, y = shuffle(X, y)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print("Length x_train:",len(x_train))
print("Length x_test:",len(x_test))
print("Length y_train:",len(y_train))
print("Length y_test:",len(y_test))

Length x_train: 95
Length x_test: 47
Length y_train: 95
Length y_test: 47


In [319]:
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import label_binarize
import plotly.express as px

# Training mit KNN
knc = KNeighborsClassifier(8)
knc = knc.fit(x_train, y_train.ravel())

# Vorhersagen als Wahrscheinlichkeiten für jede Klass
y_pred = knc.predict_proba(x_test)

# y_test für Multiclass Labels vorbereiten
y_bin = label_binarize(y_test, [0,1,2])

# Anzahl der Klassen festlegen
n_classes = 3

# Falsepositiverate und Truepositiverate vorbereiten
fpr = {}
tpr = {}

# Für jede Klasse die True Positive und False Positive Rate bestimmen
for i in range(n_classes):
  fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], y_pred[:, i])

# Plotten

names = load_iris().target_names

fig = go.Figure()
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
for i in range(n_classes):
  fig.add_trace(go.Scatter(x=fpr[i], y=tpr[i], name=names[i], mode='lines'))

fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=700, height=500
)
fig.show()

In [321]:
from sklearn.naive_bayes import GaussianNB

gaus = GaussianNB()
gaus.fit(x_test, y_test)

# Vorhersagen als Wahrscheinlichkeiten für jede Klass
y_pred = gaus.predict_proba(x_test)

# y_test für Multiclass Labels vorbereiten
y_bin = label_binarize(y_test, [0,1,2])

# Anzahl der Klassen festlegen
n_classes = 3

# Falsepositiverate und Truepositiverate vorbereiten
fpr = {}
tpr = {}

# Für jede Klasse die True Positive und False Positive Rate bestimmen
for i in range(n_classes):
  fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], y_pred[:, i])

# Plotten
names = load_iris().target_names

fig = go.Figure()
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
for i in range(n_classes):
  fig.add_trace(go.Scatter(x=fpr[i], y=tpr[i], name=names[i], mode='lines'))

fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=700, height=500
)
fig.show()

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

